In [1]:
import nltk
import wikipedia

In [2]:
def named_entity_recognition(text):
    data = nltk.word_tokenize(text)
    data = nltk.pos_tag(data)
    data = nltk.ne_chunk(data)
    data = list(data)
    
    final = False
    
    for x in data:
        if type(x) is nltk.tree.Tree:
            final = x[0][0]
            
    return final

In [3]:
d = named_entity_recognition("Can you give me some info about Cahrles")

In [4]:
wikipedia.search("Bulgaria")

['Bulgaria',
 'Demographics of Bulgaria',
 'Sofia',
 'Bulgaria Air',
 "People's Republic of Bulgaria",
 'Balkan Wars',
 'Volga Bulgaria',
 'List of Bulgarian consorts',
 'Georgi Markov',
 'Provinces of Bulgaria']

In [7]:
bg = wikipedia.page("Bulgaria")

In [8]:
from gensim.summarization.summarizer import summarize

In [12]:
summarize(bg.content, ratio=0.01)

'Thracians, one of the three primary ancestral groups of modern Bulgarians, began appearing in the region during the Iron Age. In the late 6th century BC, the Persians conquered most of present-day Bulgaria.\nIt was superseded by the subsequent Treaty of Berlin, signed on 13 July, which provided for a much smaller state comprising Moesia and the region of Sofia, leaving large populations of Bulgarians outside the new country.\nFrom a largely agricultural country with a predominantly rural population in 1948, by the 1980s Bulgaria had transformed into an industrial economy with scientific and technological research at the top of its budgetary expenditure priorities.\nNine historical and natural objects have been inscribed in the list of UNESCO World Heritage Sites: the Madara Rider, the Thracian tombs in Sveshtari and Kazanlak, the Boyana Church, the Rila Monastery, the Rock-hewn Churches of Ivanovo, Pirin National Park, Sreburna Nature Reserve and the ancient city of Nesebar.'

In [37]:
len(bg.content[:1*10**3])

1000

In [38]:
len(bg.content)

52454

In [130]:
a = wikipedia.random(pages=10)

In [133]:
wikipedia.page(a[3]).content

'The Queen of Sheba is a Biblical and Quranic figure. The tale of her visit to King Solomon has undergone extensive Jewish, Islamic, and Ethiopian elaborations, and has become the subject of one of the most widespread and fertile cycles of legends in the Orient.\n\n\n== Narratives ==\n\n\n=== Bible ===\n\nThe queen of Sheba (מַלְכַּת־שְׁבָא\u202c, "malkat-šəḇā" in the Hebrew Bible, βασίλισσα Σαβὰ in the Septuagint, Syriac ܡܠܟܬ ܫܒܐ, Ethiopic ንግሥተ፡ሳባእ፡) came to Jerusalem "with a very great retinue, with camels bearing spices, and very much gold, and precious stones" (I Kings 10:2). "Never again came such an abundance of spices" (10:10; II Chron. 9:1–9) as those she gave to Solomon. She came "to prove him with hard questions," which Solomon answered to her satisfaction. They exchanged gifts, after which she returned to her land.\nThe use of the term ḥiddot or "riddles" (I Kings 10:1), an Aramaic loanword whose shape points to a sound shift no earlier than the sixth century B.C., indicates

In [145]:
from elasticsearch import Elasticsearch
from datetime import datetime

In [146]:
es = Elasticsearch()
INDEX_NAME = "test"

if es.indices.exists(INDEX_NAME):
    print("deleting '%s' index..." % (INDEX_NAME))
    res = es.indices.delete(index = INDEX_NAME)
    print(" response: '%s'" % (res))

request_body = {
    "settings" : {
        "number_of_shards" : 1,
        "analysis": {
            "filter": {
                "english_stop": {
                    "type": "stop",
                    "stopwords": "_english_" 
                },
                "english_keywords": {
                    "type": "keyword_marker",
                    "keywords": ["example"] 
                },
                "english_stemmer": {
                    "type": "stemmer",
                    "language": "english"
                },
                "english_possessive_stemmer": {
                    "type": "stemmer",
                  "language": "possessive_english"
                }
            },
            "analyzer": {
                "english": {
                    "tokenizer":  "standard",
                    "filter": [
                        "english_possessive_stemmer",
                        "lowercase",
                        "english_stop",
                        "english_keywords",
                        "english_stemmer"
                    ]
                }
            }
        }
    },
    "mappings" : {
        "blog" : {
            "properties" : {
                "title": {"type" : "text"},
                "text": {"type" : "text"},
                "date": {"type": "date"}
            }
        }
    },
}

print("creating '%s' index..." % (INDEX_NAME))
res = es.indices.create(index = INDEX_NAME, body = request_body)
print(" response: '%s'" % (res))

deleting 'test' index...
 response: '{'acknowledged': True}'
creating 'test' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test'}'


In [147]:
for i in range(5):
    doc = {
        'title': 'victor',
        'text': 'Hello World',
        "date": datetime.now()
    }

    res = es.index(index=INDEX_NAME, doc_type="blog", id=i, body=doc)

In [148]:
es.get(index=INDEX_NAME, doc_type="blog", id=4, )

{'_id': '4',
 '_index': 'test',
 '_source': {'date': '2018-03-17T18:41:47.028637',
  'text': 'Hello World',
  'title': 'victor'},
 '_type': 'blog',
 '_version': 1,
 'found': True}

In [149]:
es.indices.refresh(index=INDEX_NAME)

{'_shards': {'failed': 0, 'successful': 1, 'total': 2}}

In [150]:
body = {
    "query": {
        "more_like_this" : {
            "fields" : ["text"],
            "like" : "Hello",
            "min_term_freq" : 1,
            "max_query_terms" : 12
        }
    }
}
res = es.search(index=INDEX_NAME, body=body)
# print(res)
print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit)
    
print(res)

Got 5 Hits:
{'_index': 'test', '_type': 'blog', '_id': '0', '_score': 0.087011375, '_source': {'title': 'victor', 'text': 'Hello World', 'date': '2018-03-17T18:41:46.984078'}}
{'_index': 'test', '_type': 'blog', '_id': '1', '_score': 0.087011375, '_source': {'title': 'victor', 'text': 'Hello World', 'date': '2018-03-17T18:41:46.996964'}}
{'_index': 'test', '_type': 'blog', '_id': '2', '_score': 0.087011375, '_source': {'title': 'victor', 'text': 'Hello World', 'date': '2018-03-17T18:41:47.006819'}}
{'_index': 'test', '_type': 'blog', '_id': '3', '_score': 0.087011375, '_source': {'title': 'victor', 'text': 'Hello World', 'date': '2018-03-17T18:41:47.018670'}}
{'_index': 'test', '_type': 'blog', '_id': '4', '_score': 0.087011375, '_source': {'title': 'victor', 'text': 'Hello World', 'date': '2018-03-17T18:41:47.028637'}}
{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': 5, 'max_score': 0.087011375, 'hits': [{'_index': 

In [144]:
es.get(INDEX_NAME, "blog", 2)

{'_id': '2',
 '_index': 'test',
 '_source': {'date': '2018-03-17T18:15:45.231983',
  'text': 'Hello World',
  'title': 'victor'},
 '_type': 'blog',
 '_version': 1,
 'found': True}